In [1]:
import pandas as pd
import numpy as np
import geopy.distance as GD

In [6]:
google_data = pd.read_csv('/Users/nateoppenheimer/code/willbanny/Location-Analysis/raw_data/features/google_data.csv')
# google_data = google_data[['lat', 'lng']]
google_data

,lat,lng,feature_name,district
0,52.626772,-1.134946,hospital,City of Leicester (B)
1,52.648301,-1.121287,hospital,City of Leicester (B)
2,52.636918,-1.125110,hospital,City of Leicester (B)
3,52.628454,-1.120936,hospital,City of Leicester (B)
4,52.631605,-1.119917,hospital,City of Leicester (B)
...,...,...,...,...
175,52.636445,-1.134958,place_of_worship,City of Leicester (B)
176,52.636771,-1.130150,place_of_worship,City of Leicester (B)
177,52.632668,-1.133536,place_of_worship,City of Leicester (B)
178,52.635278,-1.137778,place_of_worship,City of Leicester (B)


In [7]:
example_coords = pd.DataFrame()
example_coords['lat'] = [52.626772, 52.648301, 52.636918]
example_coords['lng'] = [-1.134946, -1.121287, -1.125110]
example_coords

,lat,lng
0,52.626772,-1.134946
1,52.648301,-1.121287
2,52.636918,-1.125110


In [80]:
example_coords.iloc[0,:]

lat    52.626772
lng    -1.134946
Name: 0, dtype: float64

In [81]:
geopy.distance.geodesic([52.626772,-1.134946], [52.648301,-1.121287]).m

2567.994301568318

In [9]:
radius = 500

In [10]:
%time
example_coords.apply(lambda c1: google_data.apply(lambda c2: GD.geodesic((c1['lng'], c1['lat']), (c2['lng'], c2['lat'])).m < radius, axis=1).sum(), axis=1)




CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 13.1 µs


0     5
1     3
2    13
dtype: int64

In [50]:
example_coords = pd.read_csv('/Users/nateoppenheimer/code/willbanny/Location-Analysis/raw_data/features/bquxjob_2efec792_188918eb4d5.csv')





In [51]:
example_coords = example_coords.rename(columns={"Latitude": "lat", "Longitude": "lng"})

In [52]:
example_coords.head()

,District,Description,POLYGON_ID,GridName,lng,lat
0,City of Leicester (B),Unitary Authority,97712,Grid2_0,-1.178881,52.687005
1,City of Leicester (B),Unitary Authority,97712,Grid3_0,-1.164038,52.687005
2,City of Leicester (B),Unitary Authority,97712,Grid5_0,-1.134352,52.687005
3,City of Leicester (B),Unitary Authority,97712,Grid4_0,-1.149195,52.687005
4,City of Leicester (B),Unitary Authority,97712,Grid2_1,-1.178888,52.678007


In [53]:
features = list(google_data['feature_name'].unique())
features

['hospital', 'train_station', 'bus_station', 'park', 'place_of_worship']

In [54]:
def ret_true_false(c1, c2, r):
    dis = GD.geodesic((c1['lng'], c1['lat']), (c2['lng'], c2['lat'])).m
    if dis < r[0] and dis > r[1]:
        return True
    return False

In [55]:
def find_given_radius(r, example_coords, google_data):
    for f in features:
        example_coords[f'{f}_{r[0]}'] = \
        example_coords.apply(lambda c1: google_data[google_data['feature_name']==f].apply\
                             (lambda c2: ret_true_false(c1, c2, r), axis=1)\
                             .sum(), axis=1)
    return example_coords

In [60]:
df = find_given_radius([1700, 1600], example_coords, google_data)

In [61]:
df.sort_values(by=['hospital_1500'], ascending=False)

,District,Description,POLYGON_ID,GridName,lng,lat,hospital_1500,train_station_1500,bus_station_1500,park_1500,...,hospital_1400,train_station_1400,bus_station_1400,park_1400,place_of_worship_1400,hospital_1700,train_station_1700,bus_station_1700,park_1700,place_of_worship_1700
56,City of Leicester (B),Unitary Authority,97712,Grid5_6,-1.134453,52.633016,10,1,29,24,...,4,1,3,10,7,3,0,1,0,2
43,City of Leicester (B),Unitary Authority,97712,Grid6_5,-1.119608,52.642014,8,1,24,9,...,2,1,23,3,13,0,0,0,4,1
53,City of Leicester (B),Unitary Authority,97712,Grid4_6,-1.149277,52.633016,7,0,2,19,...,4,0,1,5,13,1,0,2,1,6
54,City of Leicester (B),Unitary Authority,97712,Grid6_6,-1.119628,52.633016,5,1,25,19,...,0,0,8,8,12,1,0,1,2,4
66,City of Leicester (B),Unitary Authority,97712,Grid5_7,-1.134470,52.624017,5,1,18,22,...,0,1,13,14,30,2,0,7,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,City of Leicester (B),Unitary Authority,97712,Grid6_4,-1.119589,52.651012,0,0,1,7,...,0,0,0,3,0,2,0,0,1,3
28,City of Leicester (B),Unitary Authority,97712,Grid3_4,-1.164080,52.651012,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,City of Leicester (B),Unitary Authority,97712,Grid2_4,-1.178911,52.651012,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,City of Leicester (B),Unitary Authority,97712,Grid1_4,-1.193742,52.651012,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
r = [0, 500, 1000, 1500]

In [66]:
r[0:1]

[0]